In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader as web
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [2]:
# start_date = dt.datetime(2016,1,19)
# end_date = dt.datetime(2021,12,31)
# data = web.DataReader(name="TSLA", data_source='yahoo', start=start_date, end=end_date)
# print(data)

In [3]:
result = pd.read_csv(r'E:\VNM\excel_vnm.csv')
result["Date"] = pd.to_datetime(result["<DTYYYYMMDD>"], format='%Y%m%d')
result['Close'] = pd.to_numeric(result['<CloseFixed>'].astype(str)) * 1000
result['High'] = pd.to_numeric(result['<HighFixed>'].astype(str)) * 1000
result['Low'] = pd.to_numeric(result['<LowFixed>'].astype(str)) * 1000
result['Volume'] = pd.to_numeric(result['<Volume>'].astype(str))
data_finish = result[['Date','High','Low' ,'Close','Volume']].copy()
#data_finish

In [5]:
result_test = pd.read_csv(r'E:\VNM\excel_vnm_test.csv')
result_test["Date"] = pd.to_datetime(result_test["<DTYYYYMMDD>"], format='%Y%m%d')
result_test['Close'] = pd.to_numeric(result_test['<CloseFixed>'].astype(str)) * 1000
result_test['High'] = pd.to_numeric(result_test['<HighFixed>'].astype(str)) * 1000
result_test['Low'] = pd.to_numeric(result_test['<LowFixed>'].astype(str)) * 1000
result_test['Volume'] = pd.to_numeric(result_test['<Volume>'].astype(str))
test_data = result[['Date','High','Low' ,'Close','Volume']].copy()
#data_finish

In [35]:
company = "VNM"
# Start = dt.datetime(2012,1,1)
# End = dt.datetime(2021,12,31)

# data = web.DataReader(company,'yahoo',Start,End)

scaler = MinMaxScaler(feature_range=(0,1))
scaler_data = scaler.fit_transform(data_finish['Close'].values.reshape(-1,1))

prediction_days = 60

x_train = []
y_train = []

for x in range(prediction_days,len(scaler_data)):
    x_train.append(scaler_data[x-prediction_days:x,0])
    y_train.append(scaler_data[x,0])

x_train, y_train =np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1)) #prediction of the next closing value


model.compile(optimizer = 'adam',loss = 'mean_squared_error')
model.fit(x_train,y_train,epochs = 100,batch_size = 32)


# test_start = dt.datetime(2022,1,1)
# test_end = dt.datetime.now

# test_data =  web.DataReader(company,'yahoo',test_start,test_end)

total_dataset=pd.concat((data_finish['Close'], test_data['Close']), axis=0)

model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values
model_inputs = model_inputs.reshape(-1,1)
model_inputs = scaler.transform(model_inputs)


x_test = []

for x in range(prediction_days,len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x,0])

x_test = np.array(x_test)
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

prediction_prices = model.predict(x_test)
prediction_prices = scaler.inverse_transform(prediction_prices)

actual_prices = data_finish['Close']




Epoch 1/100


In [3]:
# Plot the test predictions
plt.plot(actual_prices, color = "black", label=f"Actual {company} Price")
plt.plot(prediction_prices, color="red", label=f"Predicted {company} Price")
plt.title(f"{company} Share Price")
plt.xlabel("Time")
plt.ylabel(f"{company} Share Price")
plt.legend()
plt.show()

NameError: name 'actual_prices' is not defined

In [33]:

#real_data = [model_inputs[len(model_inputs)+1-prediction_days:len(model_inputs+1), 0]]
real_data = [model_inputs[len(model_inputs)  - prediction_days:len(model_inputs + 1), 0]]
real_data = np.array(real_data)
real_data=np.reshape(real_data, (real_data.shape[0], real_data.shape[1],1))

prediction=model.predict(real_data)
prediction = scaler.inverse_transform(prediction)
print(f"Prediction: {prediction}")


Prediction: [[78940.28]]
